In [1]:
import pickle
import numpy as np
import torch
import seaborn as sns
import sys
# Imports from other modules and packages in the project
sys.path.append('../')

from src.helpers import *

/home/nerea/anaconda3/envs/sinthlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [3]:
name = 'Chewie'
date = '1007'
fold = 4
target_variable = 'vel'

In [4]:
#@title Helper functions for plotting (run this cell!)
sns.set_context("notebook")

# initialize a color palette for plotting
palette = sns.xkcd_palette(["windows blue",
                            "red",
                            "medium green",
                            "dusty purple",
                            "orange",
                            "amber",
                            "clay",
                            "pink",
                            "greyish"])

## Load pre-processed data

In [2]:
data_path = '../Data/Processed_Data/Tidy_'+name+'_'+date+'.pkl'

with open(data_path, 'rb') as file:
    tidy_df = pickle.load(file)

NameError: name 'name' is not defined

## 1 - Force Adaptation Dataset

In [22]:
# Create filtered DataFrames
baseline_df = tidy_df.loc[tidy_df['epoch'] == 'BL']
force_df = tidy_df.loc[tidy_df['epoch'] == 'AD']
wo_df = tidy_df.loc[tidy_df['epoch'] == 'WO']

# Function to split IDs into 3 equal sets
def split_ids(df):
    unique_ids = df.id.unique()
    set_length = len(unique_ids) // 3
    return [unique_ids[i * set_length:(i + 1) * set_length] for i in range(3)]

# Split IDs for each epoch
ids_base = split_ids(baseline_df)
ids_force = split_ids(force_df)
ids_wo = split_ids(wo_df)

# Function to create dataset dictionary entries
def create_dataset_entry(df, ids, name_prefix):
    return {
        f'{name_prefix}{i+1}': get_dataset(df.loc[df.id.isin(ids[i])], fold, target_variable='vel', no_outliers=False, force_data=True)[:6]
        for i in range(3)
    }

# Create datasets dictionary
datasets_adaptation = {}
datasets_adaptation.update(create_dataset_entry(baseline_df, ids_base, 'Baseline'))
datasets_adaptation.update(create_dataset_entry(force_df, ids_force, 'Force'))
datasets_adaptation.update(create_dataset_entry(wo_df, ids_wo, 'Washout'))

Train trials 36
Test trials  11
Val trials 9
We are testing the optimization method on fold  4
Train trials 36
Test trials  11
Val trials 9
We are testing the optimization method on fold  4
Train trials 36
Test trials  11
Val trials 9
We are testing the optimization method on fold  4
Train trials 43
Test trials  13
Val trials 11
We are testing the optimization method on fold  4
Train trials 43
Test trials  13
Val trials 11
We are testing the optimization method on fold  4
Train trials 43
Test trials  13
Val trials 11
We are testing the optimization method on fold  4
Train trials 45
Test trials  14
Val trials 11
We are testing the optimization method on fold  4
Train trials 45
Test trials  14
Val trials 11
We are testing the optimization method on fold  4
Train trials 45
Test trials  14
Val trials 11
We are testing the optimization method on fold  4


In [23]:
data_dir = './Data'
path_to_save_data = os.path.join(data_dir, 'Real_Data_'+'Adaptation'+'.pkl')

# Pickle the data and save it to file
with open(path_to_save_data, 'wb') as handle:
    pickle.dump(datasets_adaptation, handle, protocol=4)

## 2 - Stimulation Dataset

In [4]:
data_path = '../Data/Processed_Data/Tidy_Sansa_0504.pkl'

with open(data_path, 'rb') as file:
    df = pickle.load(file)

In [5]:
baseline_df_s = df.loc[df.type == 'BASELINE'].reset_index()
tonic_df_s = df.loc[df.type == 'TONIC'].reset_index()
stim_df_s = df.loc[df.type == 'BC-REACH'].reset_index()

In [7]:
baseline_df_s.id.nunique()

29

In [8]:
tonic_df_s.id.nunique()

35

In [9]:
stim_df_s.id.nunique()

69

In [11]:
fold = 4

In [14]:
# Function to create dataset dictionary entries
def create_dataset_entry_stim(df, ids, name_prefix, num_sets):
    return {
        f'{name_prefix}{i+1}': get_dataset(df.loc[df.id.isin(ids[i])], fold, target_variable='target_vel', no_outliers=False, force_data=False)[:6]
        for i in range(num_sets)
    }


ids_base_s = [baseline_df_s.id.unique()]
ids_tonic_s = [tonic_df_s.id.unique()]
ids_stim_s = [stim_df_s.id.unique()[:35], stim_df_s.id.unique()[35:]]

# Create datasets dictionary
datasets_stimulation = {}
datasets_stimulation.update(create_dataset_entry_stim(baseline_df_s, ids_base_s, 'Baseline', 1))
datasets_stimulation.update(create_dataset_entry_stim(tonic_df_s, ids_tonic_s, 'Tonic', 1))
datasets_stimulation.update(create_dataset_entry_stim(stim_df_s, ids_stim_s, 'Triggered', 2))

Train trials 18
Test trials  6
Val trials 5
We are testing the optimization method on fold  4
Train trials 22
Test trials  7
Val trials 6
We are testing the optimization method on fold  4
Train trials 22
Test trials  7
Val trials 6
We are testing the optimization method on fold  4
Train trials 22
Test trials  7
Val trials 5
We are testing the optimization method on fold  4


In [17]:
# Desired key order
desired_order = ['Baseline1', 'Triggered1', 'Tonic1','Triggered2']

# Reorder dictionary
ordered_datasets = {key: datasets_stimulation[key] for key in desired_order}

In [18]:
data_dir = './Data'
path_to_save_data = os.path.join(data_dir, 'Real_Data_'+'Stimulation'+'.pkl')

# Pickle the data and save it to file
with open(path_to_save_data, 'wb') as handle:
    pickle.dump(ordered_datasets, handle, protocol=4)